<a href="https://colab.research.google.com/github/davidlealo/sistema-recomendacion/blob/main/recomendacion_genero_por_artista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Recomendación de género a través de artistas

Sistema que busca identificar un género o grupo de estilos musicales para un usuario a través de una aplicación aque correrá en Google Colab o soluciones similares

### Instalar librerias

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357285 sha256=e4a85a8777a50787369e375981783f5509f8f7c9f7eec9774622b1d1b73d9670
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


### Recuperar datos

In [4]:
import os
import zipfile
import requests
import pandas as pd
from surprise import Dataset, Reader

# URL del dataset
url = "https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip"
zip_file = "hetrec2011-lastfm-2k.zip"
extract_dir = "hetrec2011-lastfm-2k"

# Descargar el dataset
if not os.path.exists(zip_file):
    print("Descargando dataset...")
    response = requests.get(url)
    with open(zip_file, 'wb') as f:
        f.write(response.content)
    print("Descarga completa.")

# Descomprimir el dataset
if not os.path.exists(extract_dir):
    print("Descomprimiendo dataset...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print("Descompresión completa.")

### Cargar cada archivo en un dataframe

In [6]:
# user_artists.dat - Relación entre usuarios y artistas
user_artists_path = os.path.join(extract_dir, 'user_artists.dat')
df_user_artists = pd.read_csv(user_artists_path, sep='\t')
print("Datos de user_artists:")
print(df_user_artists.head())

# artists.dat - Información sobre los artistas
artists_path = os.path.join(extract_dir, 'artists.dat')
df_artists = pd.read_csv(artists_path, sep='\t')
print("Datos de artists:")
print(df_artists.head())

# tags.dat - Información sobre las etiquetas
# Cargar el archivo tags.dat usando una codificación alternativa
tags_path = os.path.join(extract_dir, 'tags.dat')
try:
    df_tags = pd.read_csv(tags_path, sep='\t', encoding='latin1')  # Cambiar la codificación a latin1
    print("Datos de tags:")
    print(df_tags.head())
except UnicodeDecodeError as e:
    print(f"Error de decodificación: {e}")


# user_friends.dat - Relación de amistad entre usuarios
user_friends_path = os.path.join(extract_dir, 'user_friends.dat')
df_user_friends = pd.read_csv(user_friends_path, sep='\t')
print("Datos de user_friends:")
print(df_user_friends.head())

# user_taggedartists-timestamps.dat - Etiquetas de los usuarios a los artistas con marcas de tiempo
user_taggedartists_timestamps_path = os.path.join(extract_dir, 'user_taggedartists-timestamps.dat')
df_user_taggedartists_timestamps = pd.read_csv(user_taggedartists_timestamps_path, sep='\t')
print("Datos de user_taggedartists_timestamps:")
print(df_user_taggedartists_timestamps.head())

# user_taggedartists.dat - Etiquetas de los usuarios a los artistas
user_taggedartists_path = os.path.join(extract_dir, 'user_taggedartists.dat')
df_user_taggedartists = pd.read_csv(user_taggedartists_path, sep='\t')
print("Datos de user_taggedartists:")
print(df_user_taggedartists.head())

# Preparar los datos para Surprise (usaremos user_artists.dat para recomendaciones)

reader = Reader(rating_scale=(df_user_artists['weight'].min(), df_user_artists['weight'].max()))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'weight']], reader)

Datos de user_artists:
   userID  artistID  weight
0       2        51   13883
1       2        52   11690
2       2        53   11351
3       2        54   10300
4       2        55    8983
Datos de artists:
   id               name                                         url  \
0   1       MALICE MIZER       http://www.last.fm/music/MALICE+MIZER   
1   2    Diary of Dreams    http://www.last.fm/music/Diary+of+Dreams   
2   3  Carpathian Forest  http://www.last.fm/music/Carpathian+Forest   
3   4       Moi dix Mois       http://www.last.fm/music/Moi+dix+Mois   
4   5        Bella Morte        http://www.last.fm/music/Bella+Morte   

                                          pictureURL  
0    http://userserve-ak.last.fm/serve/252/10808.jpg  
1  http://userserve-ak.last.fm/serve/252/3052066.jpg  
2  http://userserve-ak.last.fm/serve/252/40222717...  
3  http://userserve-ak.last.fm/serve/252/54697835...  
4  http://userserve-ak.last.fm/serve/252/14789013...  
Datos de tags:
   tagID     

In [ ]:
from surprise import SVD, KNNBasic
from surprise.model_selection import train_test_split, cross_validate

# Crear un set de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

# Usar SVD para el modelo de recomendación
algo = SVD()

# Entrenar el modelo
algo.fit(trainset)

# Evaluar el modelo
predictions = algo.test(testset)

### Entrenamiento y evaluación utilizando Surprise

In [7]:
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split, cross_validate

# Preparación de los datos para Surprise
reader = Reader(rating_scale=(df_user_artists['weight'].min(), df_user_artists['weight'].max()))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'weight']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

# Usar el algoritmo SVD para crear el modelo de recomendación
modelo_svd = SVD()

# Entrenar el modelo SVD
modelo_svd.fit(trainset)

# Evaluar el modelo SVD
predictions_svd = modelo_svd.test(testset)

# Mostrar algunas predicciones SVD
print("\nAlgunas predicciones SVD:")
for prediction in predictions_svd[:5]:
    print(prediction)

# Usar el algoritmo KNNBasic para crear el modelo de recomendación
modelo_knn = KNNBasic()

# Entrenar el modelo KNNBasic
modelo_knn.fit(trainset)

# Evaluar el modelo KNNBasic
predictions_knn = modelo_knn.test(testset)

# Mostrar algunas predicciones KNNBasic
print("\nAlgunas predicciones KNNBasic:")
for prediction in predictions_knn[:5]:
    print(prediction)

# Evaluar el rendimiento de ambos modelos usando validación cruzada
print("\nEvaluación usando validación cruzada:")
cross_validate(modelo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(modelo_knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Algunas predicciones SVD:
user: 824        item: 599        r_ui = 28.00   est = 352698.00   {'was_impossible': False}
user: 642        item: 584        r_ui = 429.00   est = 352698.00   {'was_impossible': False}
user: 1170       item: 12917      r_ui = 166.00   est = 352698.00   {'was_impossible': False}
user: 283        item: 5182       r_ui = 112.00   est = 352698.00   {'was_impossible': False}
user: 1705       item: 16154      r_ui = 314.00   est = 352698.00   {'was_impossible': False}
Computing the msd similarity matrix...
Done computing similarity matrix.

Algunas predicciones KNNBasic:
user: 824        item: 599        r_ui = 28.00   est = 141.42   {'actual_k': 40, 'was_impossible': False}
user: 642        item: 584        r_ui = 429.00   est = 733.24   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1170       item: 12917      r_ui = 166.00   est = 733.24   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 283        item: 5182  

{'test_rmse': array([4757.94807771, 4263.87955665, 4860.08563447, 3715.30861067,
        4117.51826367]),
 'test_mae': array([784.18466096, 771.15420953, 762.25126245, 778.87588648,
        787.02223112]),
 'fit_time': (0.42852020263671875,
  0.42980217933654785,
  0.3741624355316162,
  0.40972208976745605,
  0.31659531593322754),
 'test_time': (1.769146203994751,
  1.5931322574615479,
  1.7697937488555908,
  2.954728364944458,
  1.7849352359771729)}

### Predicciones

In [9]:
# Realizar una predicción para un usuario específico y un artista específico usando el modelo SVD
user_id = 2  # ID del usuario
artist_id = 51  # ID del artista

# Hacer la predicción con el modelo SVD
pred_svd = modelo_svd.predict(user_id, artist_id)
print(f"Predicción SVD para el usuario {user_id} y el artista {artist_id}: {pred_svd.est}")

# Hacer la predicción con el modelo KNNBasic
pred_knn = modelo_knn.predict(user_id, artist_id)
print(f"Predicción KNNBasic para el usuario {user_id} y el artista {artist_id}: {pred_knn.est}")

Predicción SVD para el usuario 2 y el artista 51: 352698
Predicción KNNBasic para el usuario 2 y el artista 51: 3360.453283463783


### Recomendaciones para usuarios y artistas definidos por el programa de prueba

In [14]:
# Obtener todos los artistID únicos desde el DataFrame de artistas
all_artist_ids = df_artists['id'].unique()

# Obtener el ID del usuario para el cual deseas hacer la recomendación (puedes cambiar el ID)
user_id = 2  # Cambia esto por el ID del usuario que te interesa

# Predecir para todos los artistas que el usuario no ha escuchado
predictions = []
for artist_id in all_artist_ids:
    if artist_id not in df_user_artists[df_user_artists['userID'] == user_id]['artistID'].values:
        # Hacer la predicción con el modelo
        pred = modelo_svd.predict(user_id, artist_id)
        predictions.append((artist_id, pred.est))

# Ordenar las predicciones de mayor a menor
predictions.sort(key=lambda x: x[1], reverse=True)

# Mostrar las 5 mejores recomendaciones de artistas junto con su nombre y estilos de música
print("\nTop 5 recomendaciones de artistas para el usuario:")
for artist_id, rating in predictions[:5]:
    # Obtener el nombre del artista
    artist_name = df_artists[df_artists['id'] == artist_id]['name'].values[0]
    # Obtener los estilos de música del artista
    artist_tags = artist_styles.get(artist_id, "No disponible")

    print(f"Artista: {artist_name}, Predicción de rating: {rating}, Estilos: {artist_tags}")



Top 5 recomendaciones de artistas para el usuario:
Artista: MALICE MIZER, Predicción de rating: 352698, Estilos: j-rock, visual kei, gothic, japanese, weeabo, jrock, better than lady gaga
Artista: Diary of Dreams, Predicción de rating: 352698, Estilos: ambient, electronic, german, industrial, seen live, gothic, gothic rock, dark, darkwave, vocal, true goth emo
Artista: Carpathian Forest, Predicción de rating: 352698, Estilos: black metal, norwegian black metal, true norwegian black metal, very kvlt, norsk arysk metal, saxophones
Artista: Moi dix Mois, Predicción de rating: 352698, Estilos: metal, gothic metal, rock, j-rock, visual kei, gothic, japanese, gothic japanese, bazarov
Artista: Bella Morte, Predicción de rating: 352698, Estilos: gothic, gothic rock, darkwave, deathrock, covers


## Aplicación

### Versión 1

In [43]:
import random
from IPython.display import Javascript, display
from google.colab.output import eval_js
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD

# Normalizar 'weight' a una escala de 1 a 5
min_weight = df_user_artists['weight'].min()
max_weight = df_user_artists['weight'].max()

df_user_artists['rating'] = 1 + 4 * (df_user_artists['weight'] - min_weight) / (max_weight - min_weight)

# Convertir el DataFrame en el formato esperado por Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'rating']], reader)

# Entrenar el modelo con Surprise
trainset = data.build_full_trainset()
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Función para obtener recomendaciones basadas en predicciones
def get_top_artists_for_user(user_id, last_liked_artist_id=None, n=5):
    # Obtener todos los artistas que el usuario no ha escuchado
    unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])

    # Predecir ratings para estos artistas usando el modelo de Surprise
    predictions = [(artist_id, modelo_svd.predict(user_id, artist_id).est) for artist_id in unheard_artists]

    # Si hay un artista que le gustó, filtrar por similitud
    if last_liked_artist_id is not None:
        # Filtrar los artistas que tengan un score más cercano al último artista que le gustó
        similar_artists = [(artist_id, score) for artist_id, score in predictions if abs(score - modelo_svd.predict(user_id, last_liked_artist_id).est) < 0.1]
        predictions = similar_artists if similar_artists else predictions  # Si no hay similares, usar todos

    # Ordenar por rating predicho y devolver los top n
    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

# Inicializar lista de artistas recomendados para un usuario aleatorio
def inicializar_recomendaciones():
    global user_id, recommended_artists, current_index, last_liked_artist_id
    user_id = random.choice(df_user_artists['userID'].unique())  # Seleccionar un usuario al azar
    recommended_artists = get_top_artists_for_user(user_id)
    current_index = 0
    last_liked_artist_id = None  # Reiniciar el último artista que le gustó

inicializar_recomendaciones()  # Generar la primera recomendación

liked_artists = []  # Lista para almacenar artistas que le gustan
like_count = 0  # Contador de "me gusta"

def create_buttons(artist_name):
    artist_name_escaped = artist_name.replace("'", "\\'").replace('"', '\\"')  # Escapar caracteres
    display(Javascript(f'''
        var div = document.createElement('div');
        var btn1 = document.createElement('button');
        btn1.innerHTML = 'Me gusta: {artist_name_escaped}';
        btn1.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['Me gusta', '{artist_name_escaped}'], {{}});
        }};

        var btn2 = document.createElement('button');
        btn2.innerHTML = 'No me gusta: {artist_name_escaped}';
        btn2.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['No me gusta', '{artist_name_escaped}'], {{}});
        }};

        div.appendChild(btn1);
        div.appendChild(btn2);
        document.body.appendChild(div);
    '''))

def handle_choice(choice, artist_name):
    global user_id, recommended_artists, current_index, last_liked_artist_id, liked_artists, like_count

    print(f'Has elegido: {choice} para {artist_name}')

    if choice == 'Me gusta':
        last_liked_artist_id = df_artists[df_artists['name'] == artist_name]['id'].values[0]  # Guardar el último artista que le gustó
        liked_artists.append(artist_name)  # Agregar el artista a la lista de artistas que le gustan
        like_count += 1  # Incrementar el contador de "me gusta"

        # Verificar si ha alcanzado 6 "me gusta"
        if like_count >= 6:
            print("Te gustan los siguientes artistas:")
            for artist in liked_artists:
                print(artist)
            return  # Termina el programa

        current_index += 1
        if current_index < len(recommended_artists):
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
        else:
            print("No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.")
            inicializar_recomendaciones()
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Nuevo usuario seleccionado. Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
    else:
        # Seleccionar un nuevo artista si elige 'No me gusta'
        unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])
        next_artist_id = random.choice(list(unheard_artists))
        next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
        next_score = modelo_svd.predict(user_id, next_artist_id).est
        print(f"Nuevo artista recomendado: {next_artist_name}, score: {next_score}")
        create_buttons(next_artist_name)

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.callback', handle_choice)

# Mostrar el primer artista recomendado
first_artist_id, first_score = recommended_artists[current_index]
first_artist_name = df_artists[df_artists['id'] == first_artist_id]['name'].values[0]
print(f"Artista recomendado: {first_artist_name}, score: {first_score}")
create_buttons(first_artist_name)


Artista recomendado: Howie Day, score: 1.3132620569289257


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Howie Day
Nuevo artista recomendado: Intizar, score: 1.0424337977337643


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Intizar
Nuevo artista recomendado: Tardiss, score: 1.083776007352242


...

Has elegido: No me gusta para 藤本美貴
Nuevo artista recomendado: Junya Nakano, Masashi Hamauzu & Nobuo Uematsu, score: 1.0990235797971648


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Junya Nakano, Masashi Hamauzu & Nobuo Uematsu
No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.
Nuevo usuario seleccionado. Artista recomendado: William Fitzsimmons, score: 1.4303289051738701


<IPython.core.display.Javascript object>

Has elegido: Me gusta para William Fitzsimmons
Te gustan los siguientes artistas:
Technotronic
The Hood Internet
Kollektiv Turmstrasse
Amon Düül II
Junya Nakano, Masashi Hamauzu & Nobuo Uematsu
William Fitzsimmons
Has elegido: Me gusta para William Fitzsimmons
Te gustan los siguientes artistas:
Technotronic
The Hood Internet
Kollektiv Turmstrasse
Amon Düül II
Junya Nakano, Masashi Hamauzu & Nobuo Uematsu
William Fitzsimmons
William Fitzsimmons


### Versión 2

In [45]:
import random
from IPython.display import Javascript, display, HTML
from google.colab.output import eval_js
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD

# Normalizar 'weight' a una escala de 1 a 5
min_weight = df_user_artists['weight'].min()
max_weight = df_user_artists['weight'].max()

df_user_artists['rating'] = 1 + 4 * (df_user_artists['weight'] - min_weight) / (max_weight - min_weight)

# Convertir el DataFrame en el formato esperado por Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'rating']], reader)

# Entrenar el modelo con Surprise
trainset = data.build_full_trainset()
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Función para obtener recomendaciones basadas en predicciones
def get_top_artists_for_user(user_id, last_liked_artist_id=None, n=5):
    # Obtener todos los artistas que el usuario no ha escuchado
    unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])

    # Predecir ratings para estos artistas usando el modelo de Surprise
    predictions = [(artist_id, modelo_svd.predict(user_id, artist_id).est) for artist_id in unheard_artists]

    # Si hay un artista que le gustó, filtrar por similitud
    if last_liked_artist_id is not None:
        # Filtrar los artistas que tengan un score más cercano al último artista que le gustó
        similar_artists = [(artist_id, score) for artist_id, score in predictions if abs(score - modelo_svd.predict(user_id, last_liked_artist_id).est) < 0.1]
        predictions = similar_artists if similar_artists else predictions  # Si no hay similares, usar todos

    # Ordenar por rating predicho y devolver los top n
    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

# Inicializar lista de artistas recomendados para un usuario aleatorio
def inicializar_recomendaciones():
    global user_id, recommended_artists, current_index, last_liked_artist_id
    user_id = random.choice(df_user_artists['userID'].unique())  # Seleccionar un usuario al azar
    recommended_artists = get_top_artists_for_user(user_id)
    current_index = 0
    last_liked_artist_id = None  # Reiniciar el último artista que le gustó

inicializar_recomendaciones()  # Generar la primera recomendación

liked_artists = []  # Lista para almacenar artistas que le gustan
like_count = 0  # Contador de "me gusta"

def create_buttons(artist_name):
    artist_name_escaped = artist_name.replace("'", "\\'").replace('"', '\\"')  # Escapar caracteres
    display(Javascript(f'''
        var div = document.createElement('div');
        var btn1 = document.createElement('button');
        btn1.innerHTML = 'Me gusta: {artist_name_escaped}';
        btn1.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['Me gusta', '{artist_name_escaped}'], {{}});
        }};

        var btn2 = document.createElement('button');
        btn2.innerHTML = 'No me gusta: {artist_name_escaped}';
        btn2.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['No me gusta', '{artist_name_escaped}'], {{}});
        }};

        div.appendChild(btn1);
        div.appendChild(btn2);
        document.body.appendChild(div);
    '''))

def handle_choice(choice, artist_name):
    global user_id, recommended_artists, current_index, last_liked_artist_id, liked_artists, like_count

    print(f'Has elegido: {choice} para {artist_name}')

    if choice == 'Me gusta':
        last_liked_artist_id = df_artists[df_artists['name'] == artist_name]['id'].values[0]  # Guardar el último artista que le gustó
        liked_artists.append(artist_name)  # Agregar el artista a la lista de artistas que le gustan
        like_count += 1  # Incrementar el contador de "me gusta"

        # Verificar si ha alcanzado 6 "me gusta"
        if like_count >= 6:
            # Mostrar mensaje final en rojo
            final_message = "<h2 style='color: red;'>Te gustan los siguientes artistas:</h2><ul>"
            final_message += ''.join(f"<li style='color: red;'>{artist}</li>" for artist in liked_artists)
            final_message += "</ul>"
            display(HTML(final_message))
            return  # Termina el programa

        current_index += 1
        if current_index < len(recommended_artists):
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
        else:
            print("No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.")
            inicializar_recomendaciones()
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Nuevo usuario seleccionado. Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
    else:
        # Seleccionar un nuevo artista si elige 'No me gusta'
        unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])
        next_artist_id = random.choice(list(unheard_artists))
        next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
        next_score = modelo_svd.predict(user_id, next_artist_id).est
        print(f"Nuevo artista recomendado: {next_artist_name}, score: {next_score}")
        create_buttons(next_artist_name)

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.callback', handle_choice)

# Mostrar el primer artista recomendado
first_artist_id, first_score = recommended_artists[current_index]
first_artist_name = df_artists[df_artists['id'] == first_artist_id]['name'].values[0]
print(f"Artista recomendado: {first_artist_name}, score: {first_score}")
create_buttons(first_artist_name)


Artista recomendado: Gravity Kills, score: 1.3552713191529386


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Gravity Kills
Artista recomendado: Blessure Grave, score: 1.3552705961018514


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Blessure Grave
Artista recomendado: Sr. Chinarro, score: 1.3223203416237639


...

Has elegido: Me gusta para Lemon Demon
Artista recomendado: Daniela Herrero, score: 1.317804938313863


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Daniela Herrero
No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.
Nuevo usuario seleccionado. Artista recomendado: Giorgia Fumanti, score: 1.4163518936387356


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Giorgia Fumanti


### Versión 3

In [49]:
import random
from IPython.display import Javascript, display, HTML
from google.colab.output import eval_js
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Normalizar 'weight' a una escala de 1 a 5
min_weight = df_user_artists['weight'].min()
max_weight = df_user_artists['weight'].max()

df_user_artists['rating'] = 1 + 4 * (df_user_artists['weight'] - min_weight) / (max_weight - min_weight)

# Continuar con Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'rating']], reader)

# Entrenar el modelo con Surprise
trainset = data.build_full_trainset()
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Función para obtener recomendaciones basadas en predicciones
def get_top_artists_for_user(user_id, n=5):
    # Obtener todos los artistas que el usuario no ha escuchado
    unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])

    # Predecir ratings para estos artistas usando el modelo de Surprise
    predictions = [(artist_id, modelo_svd.predict(user_id, artist_id).est) for artist_id in unheard_artists]

    # Ordenar por rating predicho y devolver los top n
    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

# Inicializar lista de artistas recomendados para un usuario aleatorio
def inicializar_recomendaciones():
    global user_id, recommended_artists, current_index
    user_id = random.choice(df_user_artists['userID'].unique())  # Seleccionar un usuario al azar
    recommended_artists = get_top_artists_for_user(user_id)
    current_index = 0

inicializar_recomendaciones()  # Generar la primera recomendación

# Variables para almacenar artistas que le gustan
liked_artists = []
max_likes = 6

def create_buttons(artist_name):
    artist_name_escaped = artist_name.replace("'", "\\'").replace('"', '\\"')  # Escapar caracteres
    display(Javascript(f'''
        var div = document.createElement('div');
        var btn1 = document.createElement('button');
        btn1.innerHTML = 'Me gusta: {artist_name_escaped}';
        btn1.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['Me gusta', '{artist_name_escaped}'], {{}});
        }};

        var btn2 = document.createElement('button');
        btn2.innerHTML = 'No me gusta: {artist_name_escaped}';
        btn2.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['No me gusta', '{artist_name_escaped}'], {{}});
        }};

        div.appendChild(btn1);
        div.appendChild(btn2);
        document.body.appendChild(div);
    '''))

def handle_choice(choice, artist_name):
    global user_id, recommended_artists, current_index, liked_artists

    print(f'Has elegido: {choice} para {artist_name}')

    if choice == 'Me gusta':
        liked_artists.append(artist_name)  # Agregar a la lista de artistas que le gustan
        current_index += 1
        if len(liked_artists) >= max_likes:
            print(HTML("<span style='color: red;'>Artistas que te gustan: {}</span>".format(', '.join(liked_artists))))
            print(HTML("<span style='color: yellow;'>Estilos musicales favoritos:</span>"))
            display_favorite_tags()  # Llamar a la función para mostrar los estilos favoritos
            return  # Terminar el programa si se alcanzan los 6 me gusta

        if current_index < len(recommended_artists):
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
        else:
            print("No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.")
            inicializar_recomendaciones()
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Nuevo usuario seleccionado. Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
    else:
        # Seleccionar un nuevo artista similar si elige 'No me gusta'
        unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])
        next_artist_id = random.choice(list(unheard_artists))
        next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
        next_score = modelo_svd.predict(user_id, next_artist_id).est
        print(f"Nuevo artista recomendado: {next_artist_name}, score: {next_score}")
        create_buttons(next_artist_name)

# Función para mostrar estilos musicales favoritos
def display_favorite_tags():
    # Obtener los artistIDs de los artistas que le gustan
    liked_artist_ids = df_user_artists[df_user_artists['artistID'].isin(liked_artists)]['artistID'].unique()
    print(f"Artist IDs que le gustan: {liked_artist_ids}")  # Impresión de depuración

    # Obtener los tagIDs de esos artistas
    liked_artist_tags = df_user_taggedartists_timestamps[df_user_taggedartists_timestamps['artistID'].isin(liked_artist_ids)]['tagID']
    print(f"Tag IDs asociados a artistas que le gustan: {liked_artist_tags}")  # Impresión de depuración

    # Obtener los tagValues
    favorite_tags = df_tags[df_tags['tagID'].isin(liked_artist_tags)]
    print(f"Tags obtenidos: {favorite_tags}")  # Impresión de depuración

    # Imprimir los estilos musicales favoritos en amarillo
    favorite_tags_list = favorite_tags['tagValue'].tolist()
    if favorite_tags_list:
        print(HTML(f"<span style='color: yellow;'>Estilos musicales favoritos: {', '.join(favorite_tags_list)}</span>"))
    else:
        print("No se encontraron estilos musicales favoritos.")

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.callback', handle_choice)

# Mostrar el primer artista recomendado
first_artist_id, first_score = recommended_artists[current_index]
first_artist_name = df_artists[df_artists['id'] == first_artist_id]['name'].values[0]
print(f"Artista recomendado: {first_artist_name}, score: {first_score}")
create_buttons(first_artist_name)


Artista recomendado: Model 500, score: 1.4116368229262912


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Model 500
Nuevo artista recomendado: Legacy Five, score: 1


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Legacy Five
Nuevo artista recomendado: Coiffeur, score: 1.0603253552862528


...

Has elegido: Me gusta para OPM
Artista recomendado: Boney neM, score: 1.329109320000075


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Boney neM
No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.
Nuevo usuario seleccionado. Artista recomendado: Them Crooked Vultures, score: 1.3410154468252984


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Them Crooked Vultures
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
Artist IDs que le gustan: []
Tag IDs asociados a artistas que le gustan: Series([], Name: tagID, dtype: int64)
Tags obtenidos: Empty DataFrame
Columns: [tagID, tagValue]
Index: []
No se encontraron estilos musicales favoritos.
